**What does this notebook do?**
- Load the exported CGM values from NutriSense
- Print out what days are included in the dataset
- Filter down CGM data to only one day
- Extract data for meals and exercise
- Interpolate missing data and smooth curve
- Plot CGM data for that day

In [ ]:
import pandas as pd
import plotly.express as px
import datetime

# Read in CSV file
df = pd.read_csv('export.csv')

# Remove "time zone offset" from "occurred_at" column and add new "occurred_at_day" column
df['occurred_at_day'] = df['occurred_at'].apply(lambda x: x[:len(x) - 15])
df['occurred_at'] = df['occurred_at'].apply(lambda x: x[:len(x) - 6])
df.head()

In [ ]:
# Print all days with data
daysWithData = df['occurred_at_day'].unique()
print(daysWithData)

In [ ]:
# Filter down to one day, pick the second day in the dataset
day = daysWithData[2]
df = df[df['occurred_at_day']==day]

# Create a datasets just with glucose measurments
gm = df[df['class']=='GlucoseMeasurement']

# Create a dataset for meals and exercise, sort it
meals_and_exercise = df[((df['class']=='Meal') | (df['class']=='ExerciseActivity') )]
sorted_meals_and_exercise = meals_and_exercise.sort_values(by=["occurred_at"], ascending=True)

# Only keep the columns that we need
gm_data = gm.filter(['occurred_at', 'value'])

# rename the columns for easier readability
gm_data.columns = ['time', 'value']

# turn time column into the index and delete time column
gm_data['time']= pd.to_datetime(gm_data['time'])
gm_data.index = gm_data['time']
del gm_data['time']

gm_data = gm_data.resample('1T').mean() # add rows for every 1 minute
gm_data = gm_data.interpolate(method='cubic') # interpolate the new 1 minute points with data

In [ ]:
# Chart the data
fig = px.line(gm_data, y="value")

# Add data for meals and exercise
yText = 60
eventColor = "green"
for index, row in sorted_meals_and_exercise.iterrows():
    if (row['class'] == "Meal"):
        eventColor = "black"
    else:
        eventColor = "green"
    time = datetime.datetime.strptime(row['occurred_at'], '%Y-%m-%d %H:%M:%S')
    # draw a vertical line at the time of the meal/exercise
    fig.add_shape(type="line", xref="x", yref="y", x0=time, y0=70, x1=time , y1=140, line_color=eventColor,)
    # Alternate text placement so adjacent text doesn't overlap
    if (yText == 60): yText = 65
    else: yText = 60
    fig.add_annotation(text=row['description'], xref="x", yref="y", x=time, y=yText, showarrow=False, font=dict(color=eventColor))

# Add axis titles
fig.update_xaxes(title_text=str(day), tickformat='%H:%M')
fig.update_yaxes(title_text='mg/dL')
fig.show()